# MemoryOS Forking Logic Verification

This notebook verifies that:
1. **Memory isolation on init**: When an agent is initialized (like in `run_sct_hangman.py`), its memory is empty
2. **Branch memory isolation**: Memory does not leak across branches after forking


In [1]:
# Cell 1: Setup - Load environment and imports
import os
import sys
import shutil
from dotenv import load_dotenv

# Add project root to path
sys.path.insert(0, os.path.abspath('..'))

load_dotenv()

# Verify API key
assert os.getenv("OPENROUTER_API_KEY"), "OPENROUTER_API_KEY not found in environment"

from langchain_core.messages import HumanMessage, AIMessage
from hangman.providers.llmprovider import load_llm_provider
from hangman.agents.memoryos_agent import MemoryOSAgent

# Config paths (same as run_sct_hangman.py)
PROVIDERS_CONFIG = "../config/config.yaml"
MEMORYOS_CONFIG = "../config/memoryos_config_gptoss_20b.yaml"

# Clean up any previous test data
TEST_DATA_DIR = "./memoryos_fork_test_data"
if os.path.exists(TEST_DATA_DIR):
    shutil.rmtree(TEST_DATA_DIR)

print("✅ Setup complete")


/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/letta/schemas/letta_message.py:179: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class ToolCallMessage(LettaMessage):
/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:319: PydanticDeprecatedSince20: `json_encoders` is deprecated. See https://docs.pydantic.dev/2.12/concepts/serialization/#custom-serializers for alternatives. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  warnings.warn(
/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/letta/schemas/response_format.py:18: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be r

faiss.loader - INFO - Loading faiss with AVX2 support.
faiss.loader - INFO - Successfully loaded faiss with AVX2 support.
✅ Setup complete


/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/lightmem/configs/logging/base.py:7: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class LoggingConfig(BaseModel):


In [2]:
# Cell 2: Test 1 - Verify fresh agent has empty memory (like run_sct_hangman.py)

# Load LLM provider (same pattern as run_sct_hangman.py)
llm_provider = load_llm_provider(PROVIDERS_CONFIG, "gpt_oss_20b_openrouter")

# Create agent with trial-specific session_id (same pattern as run_sct_hangman.py)
session_prefix = "gpt_oss_20b_openrouter_hangman_sct_memoryos_agent"
trial_index = 1
trial_session_id = f"{session_prefix}_trial_{trial_index}"

# Override data storage path for testing
agent = MemoryOSAgent(
    llm_provider=llm_provider,
    memoryos_config={
        "llm_model": "openai/gpt-oss-20b",
        "data_storage_path": f"{TEST_DATA_DIR}/{trial_session_id}",
    },
    session_id=trial_session_id,
    m_recent=10,
    short_term_capacity=10,
    mid_term_heat_threshold=5,
    k_retrieve=7,
)

# Verify memory is empty on fresh init
private_state = agent.get_private_state()
print("=== Fresh Agent Private State ===")
print(private_state)
print()

# Check short-term memory is empty
st_memories = agent.memo.short_term_memory.get_all()
assert len(st_memories) == 0, f"Expected empty short-term memory, got {len(st_memories)} items"
print("✅ TEST 1 PASSED: Fresh agent has empty memory")


Initializing Memoryos for user 'gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1_user' and assistant 'gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1_assistant'. Data path: /network/scratch/b/baldelld/hangman/notebooks/memoryos_fork_test_data/gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1
Using unified LLM model: openai/gpt-oss-20b
ShortTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_fork_test_data/gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1/users/gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1_user/short_term.json. Initializing new memory.
MidTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_fork_test_data/gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1/users/gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1_user/mid_term.json. Initializing new memory.
LongTermMemory: No history file found at /network/scratch/b/baldelld/hangman/n

In [4]:
# Cell 3: Add some memories to the main agent (simulating pre-fork turns)

messages = []

# Turn 1
messages.append(HumanMessage(content="I'm thinking of a secret word. It has 5 letters."))
response1 = agent.invoke(messages)
messages.append(AIMessage(content=response1["response"]))
print(f"Turn 1 - Agent: {response1['response'][:100]}...")

# Turn 2  
messages.append(HumanMessage(content="The word contains the letter 'A'."))
response2 = agent.invoke(messages)
messages.append(AIMessage(content=response2["response"]))
print(f"Turn 2 - Agent: {response2['response'][:100]}...")

# Verify main agent now has memories
st_memories = agent.memo.short_term_memory.get_all()
print(f"\n=== Main Agent After Pre-Fork Turns ===")
print(f"Short-term memories: {len(st_memories)}")
print(f"Sliding window size: {len(agent._window)}")
assert len(st_memories) > 0, "Main agent should have memories after conversation"
print("\n✅ Main agent has accumulated memories")


--- From OpenRouter Client ---
Wire messages: [{'role': 'system', 'content': "You are a helpful assistant with hierarchical long-term memory.\n\n# INSTRUCTIONS\nYou have access to retrieved memories from three tiers:\n1. **Recent Interactions**: Your most recent exchanges with the user\n2. **User Profile**: Accumulated knowledge about the user's preferences and traits\n3. **Learned Knowledge**: Facts and information you've learned across sessions\n\nUse these memories to:\n- Maintain consistency with your previous statements and decisions\n- Recall relevant context from prior interactions\n- Avoid contradicting yourself or repeating information unnecessarily\n- Inform your responses naturally without explicitly listing memories\n\nIf no relevant memory applies, proceed as usual while staying consistent with any task instructions.\n\n# RETRIEVED MEMORIES\n## Recent Interactions\n- User: I'm thinking of a secret word. It has 5 letters.\n  Assistant: Got it! Let’s play a quick word‑guessi

In [5]:
# Cell 4: Create two branch agents (simulating SCT forking)

# Get session config for branching (same pattern as engine_sct_hangman.py)
config = agent.get_session_config()

# Override data storage path for branches
config["memoryos_config"] = {
    "llm_model": "openai/gpt-oss-20b",
    "data_storage_path": f"{TEST_DATA_DIR}/branch_a",
}

# Create Branch A
branch_a = MemoryOSAgent(
    branch_id="branch_a",
    parent_session_id=config["session_id"],
    **config
)
branch_a.clone_memories_from(agent.memo)
for msg in agent.get_sliding_window_state():
    branch_a._window.append(msg)

# Create Branch B with different storage path
config["memoryos_config"] = {
    "llm_model": "openai/gpt-oss-20b",
    "data_storage_path": f"{TEST_DATA_DIR}/branch_b",
}
branch_b = MemoryOSAgent(
    branch_id="branch_b",
    parent_session_id=config["session_id"],
    **config
)
branch_b.clone_memories_from(agent.memo)
for msg in agent.get_sliding_window_state():
    branch_b._window.append(msg)

print("=== Branch Agents Created ===")
print(f"Branch A session_id: {branch_a.session_id}")
print(f"Branch B session_id: {branch_b.session_id}")
print(f"Branch A sliding window: {len(branch_a._window)} messages")
print(f"Branch B sliding window: {len(branch_b._window)} messages")
print(f"Branch A short-term memories: {len(branch_a.memo.short_term_memory.get_all())}")
print(f"Branch B short-term memories: {len(branch_b.memo.short_term_memory.get_all())}")
print("\n✅ Branch agents created with cloned memories")


Initializing Memoryos for user 'gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1__branch_branch_a_user' and assistant 'gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1__branch_branch_a_assistant'. Data path: /network/scratch/b/baldelld/hangman/notebooks/memoryos_fork_test_data/branch_a
Using unified LLM model: openai/gpt-oss-20b
ShortTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_fork_test_data/branch_a/users/gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1__branch_branch_a_user/short_term.json. Initializing new memory.
MidTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_fork_test_data/branch_a/users/gpt_oss_20b_openrouter_hangman_sct_memoryos_agent_trial_1__branch_branch_a_user/mid_term.json. Initializing new memory.
LongTermMemory: No history file found at /network/scratch/b/baldelld/hangman/notebooks/memoryos_fork_test_data/branch_a/users/gpt_oss_20b_openrouter_hangman_

In [6]:
# Cell 5: Test 2 - Verify memory isolation between branches

# Add unique memory to Branch A only
branch_a_messages = list(branch_a._window)
branch_a_messages.append(HumanMessage(content="Is your secret word 'APPLE'?"))
response_a = branch_a.invoke(branch_a_messages)
print(f"Branch A response: {response_a['response'][:100]}...")

# Add different unique memory to Branch B only
branch_b_messages = list(branch_b._window)
branch_b_messages.append(HumanMessage(content="Is your secret word 'BANANA'?"))
response_b = branch_b.invoke(branch_b_messages)
print(f"Branch B response: {response_b['response'][:100]}...")

# Check memory counts
main_memories = len(agent.memo.short_term_memory.get_all())
branch_a_memories = len(branch_a.memo.short_term_memory.get_all())
branch_b_memories = len(branch_b.memo.short_term_memory.get_all())

print(f"\n=== Memory Counts After Branch Divergence ===")
print(f"Main agent: {main_memories} memories")
print(f"Branch A: {branch_a_memories} memories")
print(f"Branch B: {branch_b_memories} memories")

# Verify branches have more memories than main (they added their own)
assert branch_a_memories > main_memories, "Branch A should have more memories than main"
assert branch_b_memories > main_memories, "Branch B should have more memories than main"

# Verify main agent wasn't affected by branch operations
main_final = len(agent.memo.short_term_memory.get_all())
assert main_final == main_memories, f"Main agent memories changed! Was {main_memories}, now {main_final}"

print("\n✅ TEST 2 PASSED: Branches have independent memories")


--- From OpenRouter Client ---
Wire messages: [{'role': 'system', 'content': "You are a helpful assistant with hierarchical long-term memory.\n\n# INSTRUCTIONS\nYou have access to retrieved memories from three tiers:\n1. **Recent Interactions**: Your most recent exchanges with the user\n2. **User Profile**: Accumulated knowledge about the user's preferences and traits\n3. **Learned Knowledge**: Facts and information you've learned across sessions\n\nUse these memories to:\n- Maintain consistency with your previous statements and decisions\n- Recall relevant context from prior interactions\n- Avoid contradicting yourself or repeating information unnecessarily\n- Inform your responses naturally without explicitly listing memories\n\nIf no relevant memory applies, proceed as usual while staying consistent with any task instructions.\n\n# RETRIEVED MEMORIES\n## Recent Interactions\n- User: I'm thinking of a secret word. It has 5 letters.\n  Assistant: Got it! Let’s play a quick word‑guessi

In [7]:
# Cell 6: Summary and Cleanup

print("="*60)
print("MEMORYOS FORKING VERIFICATION SUMMARY")
print("="*60)
print()
print("✅ Test 1: Fresh agent initialization has empty memory")
print("   - Matches run_sct_hangman.py pattern")
print("   - No memory leakage from previous experiments")
print()
print("✅ Test 2: Branch memory isolation works correctly")
print("   - Branches start with cloned pre-fork memories")
print("   - Post-fork operations don't leak between branches")
print("   - Main agent unaffected by branch operations")
print()
print("="*60)
print("ALL TESTS PASSED - MemoryOS forking logic verified!")
print("="*60)

# Cleanup test data
if os.path.exists(TEST_DATA_DIR):
    shutil.rmtree(TEST_DATA_DIR)
    print(f"\n🧹 Cleaned up test data: {TEST_DATA_DIR}")


MEMORYOS FORKING VERIFICATION SUMMARY

✅ Test 1: Fresh agent initialization has empty memory
   - Matches run_sct_hangman.py pattern
   - No memory leakage from previous experiments

✅ Test 2: Branch memory isolation works correctly
   - Branches start with cloned pre-fork memories
   - Post-fork operations don't leak between branches
   - Main agent unaffected by branch operations

ALL TESTS PASSED - MemoryOS forking logic verified!

🧹 Cleaned up test data: ./memoryos_fork_test_data
